In [1]:
import os
import re
import pandas as pd
import numpy as np

import librosa

from extract_melodic_contour import extract_peaks_and_freqs, chunks

from scipy.stats import spearmanr

import matplotlib.pyplot as plt

In [2]:
octave_MIDI = pd.read_csv('/home/jacs/Documents/DataScience/Personal/song_similarity_audio/nsynth-train/MIDI_notes.csv')

In [3]:
octave_MIDI[octave_MIDI['Octave'].apply(lambda x: x in [4,5,6])]

,Octave,C,C#,D,D#,E,F,F#,G,G#,A,A#,B
5,4,60,61,62,63,64,65,66,67,68.0,69.0,70.0,71.0
6,5,72,73,74,75,76,77,78,79,80.0,81.0,82.0,83.0
7,6,84,85,86,87,88,89,90,91,92.0,93.0,94.0,95.0


In [4]:
octave_MIDI#['C'].iloc[0]

,Octave,C,C#,D,D#,E,F,F#,G,G#,A,A#,B
0,-1,0,1,2,3,4,5,6,7,8.0,9.0,10.0,11.0
1,0,12,13,14,15,16,17,18,19,20.0,21.0,22.0,23.0
2,1,24,25,26,27,28,29,30,31,32.0,33.0,34.0,35.0
3,2,36,37,38,39,40,41,42,43,44.0,45.0,46.0,47.0
4,3,48,49,50,51,52,53,54,55,56.0,57.0,58.0,59.0
5,4,60,61,62,63,64,65,66,67,68.0,69.0,70.0,71.0
6,5,72,73,74,75,76,77,78,79,80.0,81.0,82.0,83.0
7,6,84,85,86,87,88,89,90,91,92.0,93.0,94.0,95.0
8,7,96,97,98,99,100,101,102,103,104.0,105.0,106.0,107.0
9,8,108,109,110,111,112,113,114,115,116.0,117.0,118.0,119.0


In [5]:
common_path = '/home/jacs/Documents/DataScience/Personal/'

input_path= 'song_similarity_audio/'

dummy_path = 'song_similarity/'

output_path = 'song_similarity/melodic_contour/'

# As an input there is going to be a list of youtube links that contain audio that can be
# usefull to expand and enrich the database of that particular instrument

# As an input we want a tupple, with the link as first position and instrument as second position


In [6]:
df_into = pd.read_csv(common_path+input_path+'/TinySOL/TinySOL_metadata.csv')
df_into= df_into[df_into['Needed digital retuning']]

In [7]:
df_into

,Path,Fold,Family,Instrument (abbr.),Instrument (in full),Technique (abbr.),Technique (in full),Pitch,Pitch ID,Dynamics,Dynamics ID,Instance ID,String ID (if applicable),Needed digital retuning
1,Brass/Bass_Tuba/ordinario/BTb-ord-G1-pp-N-R100...,4,Brass,BTb,Bass Tuba,ord,ordinario,G1,31,pp,0,0,NaN,True
2,Brass/Bass_Tuba/ordinario/BTb-ord-G#1-pp-N-T16...,3,Brass,BTb,Bass Tuba,ord,ordinario,G#1,32,pp,0,0,NaN,True
3,Brass/Bass_Tuba/ordinario/BTb-ord-A1-pp-N-T23d...,2,Brass,BTb,Bass Tuba,ord,ordinario,A1,33,pp,0,0,NaN,True
6,Brass/Bass_Tuba/ordinario/BTb-ord-C2-pp-N-T13d...,1,Brass,BTb,Bass Tuba,ord,ordinario,C2,36,pp,0,0,NaN,True
8,Brass/Bass_Tuba/ordinario/BTb-ord-D2-pp-N-T18u...,1,Brass,BTb,Bass Tuba,ord,ordinario,D2,38,pp,0,0,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2902,Winds/Oboe/ordinario/Ob-ord-B5-ff-N-T11u.wav,3,Winds,Ob,Oboe,ord,ordinario,B5,83,ff,4,0,NaN,True
2905,Winds/Oboe/ordinario/Ob-ord-D6-ff-N-R100u.wav,4,Winds,Ob,Oboe,ord,ordinario,D6,86,ff,4,0,NaN,True
2909,Winds/Oboe/ordinario/Ob-ord-F#6-ff-N-T12u.wav,0,Winds,Ob,Oboe,ord,ordinario,F#6,90,ff,4,0,NaN,True
2910,Winds/Oboe/ordinario/Ob-ord-G6-ff-N-T16u.wav,0,Winds,Ob,Oboe,ord,ordinario,G6,91,ff,4,0,NaN,True


In [8]:
notes = list(df_into['Pitch'].unique())

In [9]:
notes

['G1',
 'G#1',
 'A1',
 'C2',
 'D2',
 'D#2',
 'E2',
 'F2',
 'F#2',
 'G#2',
 'A2',
 'B2',
 'C3',
 'G3',
 'A#3',
 'B3',
 'F#1',
 'B1',
 'G2',
 'A#2',
 'C#3',
 'D3',
 'D#3',
 'E3',
 'G#3',
 'E4',
 'A#1',
 'C#4',
 'F#4',
 'G#4',
 'D5',
 'F5',
 'E5',
 'D#5',
 'A3',
 'D#4',
 'C#2',
 'C5',
 'C4',
 'D4',
 'F4',
 'F#5',
 'G5',
 'G#5',
 'A5',
 'F#3',
 'G4',
 'A4',
 'C#5',
 'B5',
 'C6',
 'C#6',
 'G#7',
 'B7',
 'E1',
 'F1',
 'F3',
 'A#4',
 'B4',
 'D6',
 'D#6',
 'A#5',
 'B6',
 'C7',
 'A6',
 'F#6',
 'G6',
 'C#7',
 'D7',
 'D#7',
 'E7',
 'F6',
 'G#6',
 'E6',
 'A#6']

In [23]:
scale_4 = [note for note in notes if len(set('4') & set(note)) != 0]

In [24]:
scale_4_4 = [note for note in scale_4 if len(set('#') & set(note)) == 0]

In [25]:
scale_5 = [note for note in notes if len(set('5') & set(note)) != 0]

In [26]:
scale_6 = [note for note in notes if len(set('6') & set(note)) != 0]

In [27]:
scalas = [scale_4, scale_5, scale_6]

In [28]:
for scalo in scalas:
    df_final = pd.DataFrame()
    if scalo == scale_4:
        scale_name = '4_Octave'
    elif scalo == scale_5:
        scale_name = '5_Octave'
    else:
        scale_name = '6_Octave'
    for note in scalo:
        note_dataset = note
        note_folder = re.sub(' ','_',str(note_dataset)).lower()+'/'
        df_note = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_features.csv')
        df_final = pd.concat((df_final,df_note), axis=0).reset_index(drop=True)
    df_final.to_csv(common_path+output_path + scale_name+'_features.csv')

# feature extraction

In [ ]:
df_into.iloc[0]['Path']

In [ ]:
audio, Fs = librosa.load(common_path+input_path+'/TinySOL/Brass/Bass_Tuba/ordinario/BTb-ord-G1-pp-N-R100u.wav')

## frequencies

In [ ]:
def main_frequencies():
    for kk in range(0,len(notes)):
        print(kk)
        note_dataset = notes[kk]
        note_folder = re.sub(' ','_',str(note_dataset)).lower()+'/'
        if not os.path.exists(common_path+output_path+note_folder):
            os.makedirs(common_path+output_path+note_folder)
#        output_file = 'database_{}_cepstrum_features'.format(instrument_folder)
        df_final = pd.DataFrame()
        for woko in df_into[df_into['Pitch'] == note_dataset]['Path']:
            df_final3 = pd.DataFrame()
            wavo = common_path+input_path+'/TinySOL/' + woko
            titulo = woko.split('/')[-1]
            titulo = re.sub('.wav','',str(titulo))
            audio, Fs = librosa.load(wavo)
            length = audio.shape[0] / Fs
            print(f"length = {length}s")
            print(f"num of chunks = {length/0.25}")
            print(titulo)
            chonkos = chunks(audio, Fs, 0.25)
            for chunk in chonkos:
                sp_sorted, freq_sorted, sp_final, freq_final = extract_peaks_and_freqs(chunk, Fs)
                rms = librosa.feature.rms(y=chunk, frame_length=len(chunk), hop_length=int(len(chunk)+2))
                spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=Fs, n_fft=len(chunk), hop_length=int(len(chunk)+2))
                rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=Fs, n_fft=len(chunk), hop_length=int(len(chunk)+2))
                zcr = librosa.feature.zero_crossing_rate(chunk, frame_length=len(chunk), hop_length=int(len(chunk)+2))
                df_final_2 = pd.DataFrame({'10_freq': [list(freq_sorted)[:10]], '10_peak': [list(sp_sorted)[:10]], 'note_played': [note_dataset]})
                df_final_2['rms']= rms[0,:]
                df_final_2['spec_cent']= spec_cent[0,:]
                df_final_2['rolloff']= rolloff[0,:]
                df_final_2['zcr']= zcr[0,:]
                df_final3 = pd.concat((df_final3,df_final_2), axis=0).reset_index(drop=True)
            df_final = pd.concat((df_final,df_final3), axis=0).reset_index(drop=True)
        df_final.to_csv(common_path+output_path+'/'+note_folder + note_dataset+'_freq.csv', index=False)

In [ ]:
main_frequencies()

## nsynth

In [ ]:
def main_frequencies_nsynth(examples, note):
    note_dataset = note
    note_folder = note.lower()+'/'
#    if not os.path.isfile(common_path+output_path+note_folder):
#        os.makedirs(common_path+output_path+note_folder)
#        output_file = 'database_{}_cepstrum_features'.format(instrument_folder)
    df_final = pd.DataFrame()
    print(note)
    for i in range(0,len(examples)):
        print(i)
        woko = examples[i]
        df_final3 = pd.DataFrame()
        wavo = input_folder+ woko
        titulo = str(woko)
        titulo = re.sub('.wav','',str(titulo))
        audio, Fs = librosa.load(wavo)
        length = audio.shape[0] / Fs
#        print(f"length = {length}s")
#        print(f"num of chunks = {length/0.25}")
#        print(titulo)
        chonkos = chunks(audio, Fs, 0.25)
        for chunk in chonkos:
            try:
                sp_sorted, freq_sorted, sp_final, freq_final = extract_peaks_and_freqs(chunk, Fs)
                rms = librosa.feature.rms(y=chunk, frame_length=len(chunk), hop_length=int(len(chunk)+2))
                spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=Fs, n_fft=len(chunk), hop_length=int(len(chunk)+2))
                rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=Fs, n_fft=len(chunk), hop_length=int(len(chunk)+2))
                zcr = librosa.feature.zero_crossing_rate(chunk, frame_length=len(chunk), hop_length=int(len(chunk)+2))
                df_final_2 = pd.DataFrame({'10_freq': [list(freq_sorted)[:10]], '10_peak': [list(sp_sorted)[:10]], 'note_played': [note_dataset]})
                df_final_2['rms']= rms[0,:]
                df_final_2['spec_cent']= spec_cent[0,:]
                df_final_2['rolloff']= rolloff[0,:]
                df_final_2['zcr']= zcr[0,:]
                df_final3 = pd.concat((df_final3,df_final_2), axis=0).reset_index(drop=True)
            except:
                continue
        df_final = pd.concat((df_final,df_final3), axis=0).reset_index(drop=True)
    df_final.to_csv(output_folder + note_folder + note_dataset+'_nsynth_freq.csv', index=False)

In [ ]:
notes_incomplete = octave_MIDI.columns[1:]
notes_complete = {}

input_folder = '/home/jacs/Documents/DataScience/Personal/song_similarity_audio/nsynth-train/audio/'
output_folder = '/home/jacs/Documents/DataScience/Personal/song_similarity/melodic_contour/'

plop = os.listdir('/home/jacs/Documents/DataScience/Personal/song_similarity_audio/nsynth-train/audio/')

for scale in [4,5,6]:
    for note in notes_incomplete:
        note_full = note + str(scale)
        MIDI = int(octave_MIDI[note].iloc[scale+1])
        nsynth_track = [plopo for plopo in plop if '0'+str(MIDI) == plopo[-11:-8]][:300]
        print(note_full)
        note_dataset = note_full
        note_folder = note_full.lower()+'/'
        if os.path.isfile(output_folder + note_folder + note_dataset+'_nsynth_freq.csv'):
            continue
        main_frequencies_nsynth(nsynth_track, note_full)

In [19]:
def hist_sum(freqs, sp, bins, sp_sum):
    min_max_freq = np.arange(200,2000,1)
    interval = int(len(min_max_freq)/bins)
    freq_ranges = [(min(min_max_freq[i*interval:(i+1)*interval]),max(min_max_freq[i*interval:(i+1)*interval])) for i in range(0,bins)]
#    print(freq_ranges)
    for kk in range(0,len(freqs)):
        freq = freqs[kk]
        for rangee in freq_ranges:
            ###### DUDA AQUI
            rangee_string = str(rangee)
            ###############
            ###### DUDA AQUI 2
            if freq in range(int(rangee[0]),int(rangee[1])):
                try:
                    sp_sum[rangee_string] += sp[kk]
                except:
                    sp_sum[rangee_string] = sp[kk]
            else:
                try:
                    sp_sum[rangee_string] += 0
                except:
                    sp_sum[rangee_string] = 0
            ###############
    return sp_sum

In [ ]:
plop = ['E4','E5','E6']

In [ ]:
notes = pd.read_csv('notes.csv')

In [ ]:
notes_scale = [notes[j] for j in [notes.index(i) for i in plop]]

In [ ]:
notes_scale

In [ ]:
freqos

## join databases

In [ ]:
scale_4

In [20]:
ii = 0
for note in scale_4:
    print(ii)
#    note = notes[5]
    note_folder = re.sub(' ','_',str(note)).lower()
    df_dumm = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_nsynth_freq.csv')
    df_dumm2 = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_freq.csv')
#    colors = ['b','g','r','k','c','m','y']
    df_dumm3  = pd.concat((df_dumm,df_dumm2), axis=0).reset_index(drop=True)
    df_dumm3['scale'] = [4]*len(df_dumm3)
    df_dumm3['note'] = [note]*len(df_dumm3)
    df_dumm3.to_csv(common_path+output_path+note_folder +'/' + note+'_freq_concat.csv', index=False)
    df_final = pd.DataFrame()
    for kk in range(0,len(df_dumm3)):
        note_played = df_dumm3.iloc[kk]['note_played']
        scale = df_dumm3.iloc[kk]['scale']
        rms_dumm = df_dumm3.iloc[kk]['rms']
        spec_cent_dumm = df_dumm3.iloc[kk]['spec_cent']
        rolloff_dumm = df_dumm3.iloc[kk]['rolloff']
        zcr_dumm = df_dumm3.iloc[kk]['zcr']
        sp_sum = dict()
        freqos = df_dumm3.iloc[kk]['10_freq']
        freqos = freqos.replace('[','').replace(']','').split(', ')
        try:
            freqos = [float(freqo) for freqo in freqos]
        except:
            continue
        spos = df_dumm3.iloc[kk]['10_peak']
        spos = spos.replace('[','').replace(']','').split(', ')
        spos = [float(spo) for spo in spos]
        sp_sum= hist_sum(freqos, spos, 90, sp_sum)
        df_final3 = pd.DataFrame({'freq_sp': [sp_sum], 'scale': [scale], 'note': [note], 'note_played': [note_played], 
                                 'rms': [rms_dumm],'spec_cent': [spec_cent_dumm], 'rolloff': [rolloff_dumm],
                                 'zcr': [zcr_dumm]})
        df_final = pd.concat((df_final,df_final3), axis=0).reset_index(drop=True)
    df_final.to_csv(common_path+output_path+note_folder +'/' + note+'_features.csv', index=False)
    ii += 1

0
1
2
3
4
5
6
7
8
9
10
11


In [21]:
ii = 0
for note in scale_5:
    print(ii)
#    note = notes[5]
    note_folder = re.sub(' ','_',str(note)).lower()
    df_dumm = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_nsynth_freq.csv')
    df_dumm2 = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_freq.csv')
#    colors = ['b','g','r','k','c','m','y']
    df_dumm3  = pd.concat((df_dumm,df_dumm2), axis=0).reset_index(drop=True)
    df_dumm3['scale'] = [5]*len(df_dumm3)
    df_dumm3['note'] = [note]*len(df_dumm3)
    df_dumm3.to_csv(common_path+output_path+note_folder +'/' + note+'_freq_concat.csv', index=False)
    df_final = pd.DataFrame()
    for kk in range(0,len(df_dumm3)):
        note_played = df_dumm3.iloc[kk]['note_played']
        scale = df_dumm3.iloc[kk]['scale']
        rms_dumm = df_dumm3.iloc[kk]['rms']
        spec_cent_dumm = df_dumm3.iloc[kk]['spec_cent']
        rolloff_dumm = df_dumm3.iloc[kk]['rolloff']
        zcr_dumm = df_dumm3.iloc[kk]['zcr']
        sp_sum = dict()
        freqos = df_dumm3.iloc[kk]['10_freq']
        freqos = freqos.replace('[','').replace(']','').split(', ')
        try:
            freqos = [float(freqo) for freqo in freqos]
        except:
            continue
        spos = df_dumm3.iloc[kk]['10_peak']
        spos = spos.replace('[','').replace(']','').split(', ')
        spos = [float(spo) for spo in spos]
        sp_sum= hist_sum(freqos, spos, 90, sp_sum)
        df_final3 = pd.DataFrame({'freq_sp': [sp_sum], 'scale': [scale], 'note': [note], 'note_played': [note_played], 
                                 'rms': [rms_dumm],'spec_cent': [spec_cent_dumm], 'rolloff': [rolloff_dumm],
                                 'zcr': [zcr_dumm]})
        df_final = pd.concat((df_final,df_final3), axis=0).reset_index(drop=True)
    df_final.to_csv(common_path+output_path+note_folder +'/' + note+'_features.csv', index=False)
    ii += 1

0
1
2
3
4
5
6
7
8
9
10
11


In [22]:
ii = 0
for note in scale_6:
    print(ii)
#    note = notes[5]
    note_folder = re.sub(' ','_',str(note)).lower()
    df_dumm = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_nsynth_freq.csv')
    df_dumm2 = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_freq.csv')
#    colors = ['b','g','r','k','c','m','y']
    df_dumm3  = pd.concat((df_dumm,df_dumm2), axis=0).reset_index(drop=True)
    df_dumm3['scale'] = [6]*len(df_dumm3)
    df_dumm3['note'] = [note]*len(df_dumm3)
    df_dumm3.to_csv(common_path+output_path+note_folder +'/' + note+'_freq_concat.csv', index=False)
    df_final = pd.DataFrame()
    for kk in range(0,len(df_dumm3)):
        note_played = df_dumm3.iloc[kk]['note_played']
        scale = df_dumm3.iloc[kk]['scale']
        rms_dumm = df_dumm3.iloc[kk]['rms']
        spec_cent_dumm = df_dumm3.iloc[kk]['spec_cent']
        rolloff_dumm = df_dumm3.iloc[kk]['rolloff']
        zcr_dumm = df_dumm3.iloc[kk]['zcr']
        sp_sum = dict()
        freqos = df_dumm3.iloc[kk]['10_freq']
        freqos = freqos.replace('[','').replace(']','').split(', ')
        try:
            freqos = [float(freqo) for freqo in freqos]
        except:
            continue
        spos = df_dumm3.iloc[kk]['10_peak']
        spos = spos.replace('[','').replace(']','').split(', ')
        spos = [float(spo) for spo in spos]
        sp_sum= hist_sum(freqos, spos, 90, sp_sum)
        df_final3 = pd.DataFrame({'freq_sp': [sp_sum], 'scale': [scale], 'note': [note], 'note_played': [note_played], 
                                 'rms': [rms_dumm],'spec_cent': [spec_cent_dumm], 'rolloff': [rolloff_dumm],
                                 'zcr': [zcr_dumm]})
        df_final = pd.concat((df_final,df_final3), axis=0).reset_index(drop=True)
    df_final.to_csv(common_path+output_path+note_folder +'/' + note+'_features.csv', index=False)
    ii += 1

0
1
2
3
4
5
6
7
8
9
10
11


In [ ]:
jj = 0
plt.figure()
for note in scale_4:
#    note = notes[5]
    note_folder = re.sub(' ','_',str(note)).lower()
    df_dumm = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_nsynth_freq.csv')
#    colors = ['b','g','r','k','c','m','y']
    colors = ['y']
    for kk in range(0,len(df_dumm)):
        sp_sum = dict()
        freqos = df_dumm.iloc[kk]['10_freq']
        freqos = freqos.replace('[','').replace(']','').split(', ')
        try:
            freqos = [float(freqo) for freqo in freqos]
        except:
            continue
        spos = df_dumm.iloc[kk]['10_peak']
        spos = spos.replace('[','').replace(']','').split(', ')
        spos = [float(spo) for spo in spos]
        sp_sum= hist_sum(freqos, spos, 90, sp_sum)
        if kk == 9:
            plt.plot(list(sp_sum.keys()), list(sp_sum.values()), '.'+colors[jj] , label = note)
        else:
            try:
                plt.plot(list(sp_sum.keys()), list(sp_sum.values()), '.'+colors[jj])
            except:
                jj = 0
                plt.plot(list(sp_sum.keys()), list(sp_sum.values()), '.'+colors[jj])
        
    jj += 1
ax = plt.gca()
ax.tick_params(axis='x', labelrotation = 90)
plt.title('4th Octave')
#plt.legend()
plt.show()

In [ ]:
jj = 0
plt.figure()
for note in scale_5:
#    note = notes[5]
    note_folder = re.sub(' ','_',str(note)).lower()
    df_dumm = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_nsynth_freq.csv')
#    colors = ['b','g','r','k','c','m','y']
    colors = ['c']
    for kk in range(0,len(df_dumm)):
        sp_sum = dict()
        freqos = df_dumm.iloc[kk]['10_freq']
        freqos = freqos.replace('[','').replace(']','').split(', ')
        try:
            freqos = [float(freqo) for freqo in freqos]
        except:
            continue
        spos = df_dumm.iloc[kk]['10_peak']
        spos = spos.replace('[','').replace(']','').split(', ')
        spos = [float(spo) for spo in spos]
        sp_sum= hist_sum(freqos, spos, 90, sp_sum)
        if kk == 9:
            plt.plot(list(sp_sum.keys()), list(sp_sum.values()), '.'+colors[jj] , label = note)
        else:
            try:
                plt.plot(list(sp_sum.keys()), list(sp_sum.values()), '.'+colors[jj])
            except:
                jj = 0
                plt.plot(list(sp_sum.keys()), list(sp_sum.values()), '.'+colors[jj])
        
    jj += 1
ax = plt.gca()
ax.tick_params(axis='x', labelrotation = 90)
plt.title('5th Octave')
#plt.legend()
plt.show()

In [ ]:
jj = 0
plt.figure()
for note in scale_6:
#    note = notes[5]
    note_folder = re.sub(' ','_',str(note)).lower()
    df_dumm = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_nsynth_freq.csv')
#    colors = ['b','g','r','k','c','m','y']
    colors = ['g']
    for kk in range(0,len(df_dumm)):
        sp_sum = dict()
        freqos = df_dumm.iloc[kk]['10_freq']
        freqos = freqos.replace('[','').replace(']','').split(', ')
        try:
            freqos = [float(freqo) for freqo in freqos]
        except:
            continue
        spos = df_dumm.iloc[kk]['10_peak']
        spos = spos.replace('[','').replace(']','').split(', ')
        spos = [float(spo) for spo in spos]
        sp_sum= hist_sum(freqos, spos, 90, sp_sum)
        if kk == 9:
            plt.plot(list(sp_sum.keys()), list(sp_sum.values()), '.'+colors[jj] , label = note)
        else:
            try:
                plt.plot(list(sp_sum.keys()), list(sp_sum.values()), '.'+colors[jj])
            except:
                jj = 0
                plt.plot(list(sp_sum.keys()), list(sp_sum.values()), '.'+colors[jj])
        
    jj += 1
ax = plt.gca()
ax.tick_params(axis='x', labelrotation = 90)
plt.title('6th Octave')
#plt.legend()
plt.show()

In [ ]:
#for ii in range(0,len(notes_scale)):
#    note = notes_scale[ii]
jj = 0
plt.figure()
for note in notes_scale:
#    note = notes[5]
    note_folder = re.sub(' ','_',str(note)).lower()
    df_dumm = pd.read_csv(common_path+output_path+note_folder +'/' + note+'_freq.csv')
    colors = ['b','g','r','k','c','m','y']
    for kk in range(0,10):
        freqos = df_dumm.iloc[kk]['10_freq']
        freqos = freqos.replace('[','').replace(']','').split(', ')
        freqos = [float(freqo) for freqo in freqos]
        spos = df_dumm.iloc[kk]['10_peak']
        spos = spos.replace('[','').replace(']','').split(', ')
        spos = [float(spo) for spo in spos]
        if kk == 9:
            plt.plot(freqos, spos, '*'+colors[jj], label= note)
        else:
            try:
                plt.plot(freqos, spos, '*'+colors[jj])
            except:
                jj = 0
                plt.plot(freqos, spos, '*'+colors[jj])
        
    jj += 1
plt.legend()
plt.show()
    
#    plt.show()

## cepstrum

In [ ]:
def main_cepstrum():
    for kk in range(0,len(notes)):
        print(kk)
        instrument_dataset = notes[kk]
        instrument_folder = re.sub(' ','_',str(instrument_dataset)).lower()+'/'
        if not os.path.exists(common_path+output_path+instrument_folder):
            os.makedirs(common_path+output_path+instrument_folder)
#        output_file = 'database_{}_cepstrum_features'.format(instrument_folder)
        df_final = pd.DataFrame()
        df_final3 = pd.DataFrame()
        for woko in df_into[df_into['Pitch'] == instrument_dataset]['Path']:
            wavo = common_path+input_path+'/TinySOL/' + woko
            titulo = woko.split('/')[-1]
            titulo = re.sub('.wav','',str(titulo))
            audio, Fs = librosa.load(wavo)
            length = audio.shape[0] / Fs
            print(f"length = {length}s")
            print(titulo)
            indexoo = 0
            df_final_2 = mel_freq_cepstrum(audio, Fs, 13, kk, instrument_dataset)
            df_final3 = pd.concat((df_final3,df_final_2), axis=0).reset_index(drop=True)
        df_final = pd.concat((df_final,df_final3), axis=0).reset_index(drop=True)
        df_final.to_csv(common_path+output_path+'/'+instrument_folder + instrument_dataset+'.csv', index=False)

In [ ]:
main_cepstrum()

In [ ]:
brass_cols = ['spec_cent', 'spec_bw',
       'rolloff', 'zcr', 'mfccs_0', 'mfccs_1', 'mfccs_2', 'mfccs_3', 'mfccs_4',
       'mfccs_5', 'mfccs_6', 'mfccs_7', 'mfccs_8', 'mfccs_9', 'mfccs_10',
       'mfccs_11', 'mfccs_12']

In [ ]:
#brass_cols = list(df_instrument.columns)[3:-4]
for col in brass_cols:
    for note in notes_scale:
        note_folder = re.sub(' ','_',str(note)).lower()
        df_dumm = pd.read_csv(common_path+output_path+note_folder +'/' + note+'.csv')
#        df_dumm = df_dumm[df_dumm['note_played'] == note]
        df_dumm[col].hist(bins=20, legend =True)
    plt.title(col)
    plt.show()

In [ ]:
tinysol = ['Bass Tuba', 'French Horn', 'Trombone', 'Trumpet in C', 'Accordion', 'Cello', 
           'Contrabass', 'Viola', 'Violin', 'Alto Saxophone', 'Bassoon', 'Clarinet in Bb',
           'Flute', 'Oboe']

In [ ]:
instruments = ['Bass Tuba','French Horn','Trombone','Trumpet in C','Accordion','Cello','Contrabass','Viola','Violin','Alto Saxophone','Bassoon','Clarinet in Bb','Flute','Oboe','Guitar','Electric guitar','Acoustic guitar','Bass guitar','Tapping','Steelpan','Percussion', 'Drum and bass', 'Wind instrument', 'Bowed string instrument', 'Keyboard', 'Single Voice Singing', 'Group Singing', 'Noise']

In [ ]:
set(instruments) - set(tinysol)

In [ ]:
instruments_reduced = {'woodwind': ['Clarinet in Bb','Flute','Oboe', 'Bassoon', 'Alto Saxophone', 'Wind Instrument'],
                       'brass':['Bass Tuba','French Horn','Trombone', 'Trumpet in C'],
                       'bowed_string_instruments': ['Cello','Viola','Violin', 'Bowed string instrument'],
                       'plucked_string_instruments':['Guitar','Electric guitar','Acoustic guitar', 'Tapping', 'Bass guitar','Contrabass'],
                       'keyboard': ['Keyboard','Accordion'],
                       'percussion': ['Steelpan','Percussion', 'Drum and bass'],
                       'singing': ['Single Voice Singing', 'Group Singing'],
                       'noise': ['Noise']}

In [ ]:
brass = pd.read_csv('/home/jacs/Documents/DataScience/Personal/data/keyboard.csv')

In [ ]:
instruments = list(dict(brass.groupby(['instrument']).count()['instrument_name']).keys())

In [ ]:
brass.groupby(['instrument']).count()['instrument_name']

In [ ]:
sol_path = '/home/jacs/Documents/DataScience/Personal/song_similarity_audio/'

In [ ]:
trumpet_sol = pd.read_csv('/home/jacs/Documents/DataScience/Personal/song_similarity_audio/trumpet_in_c/database_trumpet_in_c_cepstrum_features.csv')
trumpet_youtube = pd.read_csv('/home/jacs/Documents/DataScience/Personal/data/instruments/original/trumpet_in_c.csv')

In [ ]:
def best_examples(df_stats, df_bowed):
    df_bowed_final = pd.DataFrame()
#    cols = list(df_bowed.columns[1:-4])
    cols = ['mfccs_0','mfccs_1', 'mfccs_2', 'mfccs_3', 'mfccs_4', 'mfccs_5', 'mfccs_6','mfccs_7', 'mfccs_8', 'mfccs_9', 'mfccs_10', 'mfccs_11', 'mfccs_12']
    for i in range(0,len(cols)):
        col = cols[i]
        stats = df_stats[col].describe()[['mean', 'std']].values
        if i == 0 :
            df_bowed_final = df_bowed[df_bowed[col].apply(lambda x: stats[0]-(3*stats[1]) <= x <= stats[0]+(3*stats[1]))]
            continue
        else:
            df_iter = df_bowed[df_bowed[col].apply(lambda x: stats[0]-(3*stats[1]) <= x <= stats[0]+(3*stats[1]))]
            df_bowed_final = pd.merge(df_bowed_final, df_iter, left_index=True, right_index=True)[['index_x', 'rms_x', 'spec_cent_x', 'spec_bw_x', 'rolloff_x', 'zcr_x',
                                                                                        'mfccs_0_x', 'mfccs_1_x', 'mfccs_2_x', 'mfccs_3_x', 'mfccs_4_x',
                                                                                        'mfccs_5_x', 'mfccs_6_x', 'mfccs_7_x', 'mfccs_8_x', 'mfccs_9_x',
                                                                                        'mfccs_10_x', 'mfccs_11_x', 'mfccs_12_x', 'instrument_x',
                                                                                        'note_played_x', 'instrument_name_x']]
            df_iter = pd.DataFrame()
    df_bowed_final = df_bowed_final.drop_duplicates()
    df_bowed_final.columns = ['index', 'rms', 'spec_cent', 'spec_bw', 'rolloff', 'zcr', 'mfccs_0',
       'mfccs_1', 'mfccs_2', 'mfccs_3', 'mfccs_4', 'mfccs_5', 'mfccs_6',
       'mfccs_7', 'mfccs_8', 'mfccs_9', 'mfccs_10', 'mfccs_11', 'mfccs_12',
       'instrument', 'note_played', 'instrument_name']
    df_bowed_final = df_bowed_final.reset_index(drop=True)
    return df_bowed_final

In [ ]:
trumpet_sol.rename(columns={"rms": "rmeso"}, inplace=True)

In [ ]:
df_blop = df_blop[['index', 'rms', 'spec_cent', 'spec_bw', 'rolloff', 'zcr',
       'mfccs_0', 'mfccs_1', 'mfccs_2', 'mfccs_3', 'mfccs_4', 'mfccs_5',
       'mfccs_6', 'mfccs_7', 'mfccs_8', 'mfccs_9', 'mfccs_10', 'mfccs_11',
       'mfccs_12', 'instrument', 'note_played', 'instrument_name']]

df_blop

In [ ]:
route1 = '/home/jacs/Documents/DataScience/Personal/data/instruments/original/'
for blop in instruments:
    instrument_name = re.sub(' ','_',str(blop)).lower()
    df_blop = pd.read_csv(route1+'{}.csv'.format(instrument_name))
    df_blop.rename(columns={"indstrument_name": "instrument_name"}, inplace=True)
    dF_blop = df_blop[['index', 'rms', 'spec_cent', 'spec_bw', 'rolloff', 'zcr',
       'mfccs_0', 'mfccs_1', 'mfccs_2', 'mfccs_3', 'mfccs_4', 'mfccs_5',
       'mfccs_6', 'mfccs_7', 'mfccs_8', 'mfccs_9', 'mfccs_10', 'mfccs_11',
       'mfccs_12', 'instrument', 'note_played', 'instrument_name']]
    df_blop.to_csv(route1 +'{}.csv'.format(instrument_name), index=False)
    print(instrument_name)

In [ ]:
for sol in tinysol:
    sol_lower = re.sub(' ','_',str(sol)).lower()
    print(sol_lower)
    df_sol = pd.read_csv(sol_path + sol_lower+'/'+'database_{}_cepstrum_features.csv'.format(sol_lower))
    df_non_sol = pd.read_csv('/home/jacs/Documents/DataScience/Personal/data/instruments/original/{}.csv'.format(sol_lower))
    print(df_non_sol.shape)
    df_final=best_examples(df_sol, df_non_sol)
    df_final.to_csv('/home/jacs/Documents/DataScience/Personal/data/instruments/{}.csv'.format(sol_lower))
    print(df_final.shape)

In [ ]:
cols = ['mfccs_0',
       'mfccs_1', 'mfccs_2', 'mfccs_3', 'mfccs_4', 'mfccs_5', 'mfccs_6',
       'mfccs_7', 'mfccs_8', 'mfccs_9', 'mfccs_10', 'mfccs_11', 'mfccs_12']

for i in range(0,len(cols)):
    col = cols[i]
    stats = trumpet_sol[col].describe()[['mean', 'std']].values
    print(stats)
    if i == 0 :
        df_bowed_final = trumpet_youtube[trumpet_youtube[col].apply(lambda x: stats[0]-(2*stats[1]) <= x <= stats[0]+(2*stats[1]))]
        continue
    else:
        df_iter = trumpet_youtube[trumpet_youtube[col].apply(lambda x: stats[0]-(2*stats[1]) <= x <= stats[0]+(2*stats[1]))]
        df_bowed_final = pd.merge(df_bowed_final, df_iter, left_index=True, right_index=True)[['index_x', 'rms_x', 'spec_cent_x', 'spec_bw_x', 'rolloff_x', 'zcr_x',
                                                                                    'mfccs_0_x', 'mfccs_1_x', 'mfccs_2_x', 'mfccs_3_x', 'mfccs_4_x',
                                                                                    'mfccs_5_x', 'mfccs_6_x', 'mfccs_7_x', 'mfccs_8_x', 'mfccs_9_x',
                                                                                    'mfccs_10_x', 'mfccs_11_x', 'mfccs_12_x', 'instrument_x',
                                                                                    'note_played_x']]
        df_iter = pd.DataFrame()
df_bowed_final = df_bowed_final.drop_duplicates()
#df_bowed_final.columns = ['index', 'rms', 'spec_cent', 'spec_bw', 'rolloff', 'zcr', 'mfccs_0',
#   'mfccs_1', 'mfccs_2', 'mfccs_3', 'mfccs_4', 'mfccs_5', 'mfccs_6',
#   'mfccs_7', 'mfccs_8', 'mfccs_9', 'mfccs_10', 'mfccs_11', 'mfccs_12',
#   'instrument', 'note_played', 'instrument_name']
df_bowed_final = df_bowed_final.reset_index(drop=True)

In [ ]:
df_bowed_final

In [ ]:
trumpet_youtube.shape

In [ ]:
instruments_reduced['woodwind']

In [ ]:
final_data_path = common_path + dummy_path + 'data/new/' #instrument_folder

In [ ]:
input_path = '/home/jacs/Documents/DataScience/Personal/data/instruments/original/'
output_path = '/home/jacs/Documents/DataScience/Personal/data/instruments/new/'

In [ ]:
blop = pd.read_csv('/home/jacs/Documents/DataScience/Personal/data/instruments/cello.csv')

In [ ]:
best_examples(blop)

In [ ]:
for pp in range(0,len(instruments)):
    instrument = instruments[pp]
    instrument_folder = re.sub(' ','_',str(instrument)).lower()#+'/'
    print(instrument_folder)
    df_iteration = pd.read_csv(input_path + instrument_folder+'.csv')
    df_iteration = best_examples(df_iteration)
    df_iteration.to_csv(output_path+'{}.csv'.format(instrument_folder))

In [ ]:
for pp in range(0,len(instruments)):
    if not os.path.exists(final_data_path):
        os.makedirs(final_data_path)
    df_final2 = pd.DataFrame([])
    instrument = instruments[pp]
#        print(instrument)
    instrument_folder = re.sub(' ','_',str(instrument)).lower()#+'/'
    input_data_path = common_path + input_path+ instrument_folder
#    print(os.listdir(path))
    for file in os.listdir(input_data_path):
        if file == '{}_youtube_database_enrichment.csv'.format(instrument_folder):
            continue
        else:
            print(file)
            df_iteration = pd.read_csv(input_data_path+'/'+file)
            df_final2 = pd.concat((df_final2,df_iteration), axis=0).reset_index(drop=True)
    df_final2.to_csv(final_data_path+'{}.csv'.format(instrument_folder))
#    links_audio = list(df_links['youtube_links'])
#        print(instrument,len(links_audio))
#    titles = list(df_links['title'])


In [ ]:
blop = pd.read_csv('/home/jacs/Documents/DataScience/Personal/data/instruments/accordion.csv')

In [ ]:
stats = blop['mfccs_1'].describe()[['mean', 'std']].values
mffc1 = blop[blop['mfccs_1'].apply(lambda x: stats[0]-stats[1] <= x <= stats[0]+stats[1])]

stats = blop['mfccs_2'].describe()[['mean', 'std']].values
mfcc2 = blop[blop['mfccs_2'].apply(lambda x: stats[0]-stats[1] <= x <= stats[0]+stats[1])]

stats = blop['mfccs_3'].describe()[['mean', 'std']].values
mfcc3 = blop[blop['mfccs_3'].apply(lambda x: stats[0]-stats[1] <= x <= stats[0]+stats[1])]

df_bowed_final = pd.merge(mffc1, mfcc2, left_index=True, right_index=True)[['index_x', 'rms_x', 'spec_cent_x', 'spec_bw_x', 'rolloff_x', 'zcr_x',
       'mfccs_0_x', 'mfccs_1_x', 'mfccs_2_x', 'mfccs_3_x', 'mfccs_4_x',
       'mfccs_5_x', 'mfccs_6_x', 'mfccs_7_x', 'mfccs_8_x', 'mfccs_9_x',
       'mfccs_10_x', 'mfccs_11_x', 'mfccs_12_x', 'instrument_x',
       'note_played_x', 'instrument_name_x']]

df_bowed_final = pd.merge(df_bowed_final, mfcc3, left_index=True, right_index=True)[['index_x', 'rms_x', 'spec_cent_x', 'spec_bw_x', 'rolloff_x', 'zcr_x',
       'mfccs_0_x', 'mfccs_1_x', 'mfccs_2_x', 'mfccs_3_x', 'mfccs_4_x',
       'mfccs_5_x', 'mfccs_6_x', 'mfccs_7_x', 'mfccs_8_x', 'mfccs_9_x',
       'mfccs_10_x', 'mfccs_11_x', 'mfccs_12_x', 'instrument_x',
       'note_played_x', 'instrument_name_x']]

print(mffc1.shape, mfcc2.shape, mfcc3.shape, df_bowed_final.shape)

In [ ]:
setto = set(mffc1.index) & set(mfcc2.index)

In [ ]:
pd.merge(mffc1, mfcc2, left_index=True, right_index=True)[['index_x', 'rms_x', 'spec_cent_x', 'spec_bw_x', 'rolloff_x', 'zcr_x',
       'mfccs_0_x', 'mfccs_1_x', 'mfccs_2_x', 'mfccs_3_x', 'mfccs_4_x',
       'mfccs_5_x', 'mfccs_6_x', 'mfccs_7_x', 'mfccs_8_x', 'mfccs_9_x',
       'mfccs_10_x', 'mfccs_11_x', 'mfccs_12_x', 'instrument_x',
       'note_played_x', 'instrument_name_x']]

In [ ]:
blop.columns

In [ ]:
blop.shape

In [ ]:
best_examples(blop)

In [ ]:
df_final3=df_final2

In [ ]:
blopo

In [ ]:
blopo = df_final.columns
splopo = np.zeros((len(blopo),len(blopo)))
for i in range(0,len(blopo)):
    for j in range(0,len(blopo)):
        corr, _ = spearmanr(df_final2[blopo[i]], df_final2[blopo[j]])
        splopo[i,j] = corr
#print('Spearmans correlation: %.3f' % corr)

splopo

In [ ]:
for i in range(0,len(blopo)):
    print(blopo[i],df_final2[blopo[i]].describe())

In [ ]:
for i in range(0,len(blopo)):
    print(blopo[i],df_final2[blopo[i]].describe(),df_final3[blopo[i]].describe())

In [ ]:
linko1 = '/home/jacs/Documents/DataScience/Personal/song_similarity_audio/trumpet_in_c/database_trumpet_in_c_cepstrum_features.csv'
linko2 = '/home/jacs/Documents/DataScience/Personal/song_similarity_audio/alto_saxophone/database_alto_saxophone_cepstrum_features.csv'

In [ ]:
df_final2['mfccs_0'].describe()

In [ ]:
plt.plot(df_final3['rms'],df_final3['mfccs_0'], '.b')
plt.plot(df_final2['rms'],df_final2['mfccs_0'], '+r')

In [ ]:
df_tru = pd.read_csv(linko1)
df_sax = pd.read_csv(linko2)

In [ ]:
df_tru_new = pd.DataFrame([])
df_sax_new = pd.DataFrame([])
for tuu in df_tru.columns:
    df_tru_new[tuu] = df_tru[tuu].describe()
    df_sax_new[tuu] = df_sax[tuu].describe()

In [ ]:
df_tru_new

In [ ]:
df_sax_new

In [ ]:
plt.plot(df_acc['rms'],df_acc['mfccs_0'], '.b')
plt.plot(df_sax['rms'],df_sax['mfccs_0'], '+r')

In [ ]:
path = '/home/jacs/Documents/DataScience/Personal/song_similarity_audio/'
instrument = 'cello'
os.listdir(path)

In [ ]:
# for data transformation
import numpy as np
# for visualizing the data
import matplotlib.pyplot as plt
# for opening the media file
import scipy.io.wavfile as wavfile

import re
import librosa
import scipy
# for dataframe manipulation
import pandas as pd

In [ ]:
wavo = '/home/jacs/Documents/DataScience/Personal/song_similarity_audio/TinySOL/Brass/Trumpet_C/ordinario/TpC-ord-D4-ff-N-T27u.wav'

In [ ]:
Fs, audio = wavfile.read(wavo)

In [ ]:
#mfccs = librosa.feature.mfcc(y=audio, sr=Fs)
#print(mfccs.shape)

In [ ]:
x, fs = librosa.load(wavo)
#librosa.display.waveplot(x, sr=fs)

In [ ]:
xx= x#[:5*fs]

In [ ]:
mfccs = librosa.feature.mfcc(y=xx, sr=fs)#, norm='ortho')#, n_fft=26)
print(mfccs.shape)

In [ ]:
spec_cent = librosa.feature.spectral_centroid(y=xx, sr=fs)
spec_bw = librosa.feature.spectral_bandwidth(y=xx, sr=fs)
rolloff = librosa.feature.spectral_rolloff(y=xx, sr=fs)
zcr = librosa.feature.zero_crossing_rate(xx)

In [ ]:
df = pd.DataFrame()
df['zcr']= zcr[0,:]
for i in range(0,mfccs.shape[0]):
    df['mfccs_{}'.format(i)] = mfccs[i,:]

In [ ]:
df2 = pd.DataFrame()

In [ ]:
pd.concat((df,df2), axis=0)

In [ ]:
np.mean(mfccs[:,0])

In [ ]:
plt.plot(np.log(xx))

In [ ]:
meano = []

for i in range(0,mfccs.shape[1]):
    meano.append(np.mean(mfccs[:,i]))

In [ ]:
plt.plot(meano)
#    plt.show()
#    plt.xlim(0,50)
#    plt.ylim(-350,-300)

In [ ]:
sp = scipy.fft.fft(xx)
freq = scipy.fft.fftfreq(len(xx),fs)

In [ ]:
plt.plot(freq,sp)
plt.xlim(0,0.00002)

In [ ]:
plop = len(xx)
plop2= mfccs.shape[1]

print((plop/plop2)/fs)

In [ ]:
mfccs[:10,:].shape

In [ ]:
librosa.display.specshow(mfccs, sr=fs, x_axis='time')

In [ ]:
mfccs[0]`

In [ ]:
mfccs = sklearn.preprocessing.scale(mfccs, axis=1)
print(mfccs.mean(axis=1))
print(mfccs.var(axis=1))


In [ ]:
flop = 'TOP_10_MOST_POPULAR_TRUMPET_SONGS_(with_Sheet_Music_Notes)'

In [ ]:
re.sub('[)!$/(]','',flop)

In [ ]:
Fs, audio = wavfile.read('The_Most_Beautiful_Trumpet_Solo.wav')

In [ ]:
#top_2_idx = np.argsort(sp)[-40:]
#listo_freq = [freq[i] for i in top_2_idx]

In [ ]:
# Creates different chunks of data from a signal
# Includes half the past chunk
# Say, you have a 1s signal, you want windows of 0.25s 
# with this methodology you will have each window that includes half of the past window

def chunks(xs, n):
    # xs: signal
    # n: times you want to split the signal
    # len(return): n(even): 2n - 1
    #            : n(odd): 2n +1
    ms = int(len(xs)/n)
    return list(xs[i-int(ms/2):i+int(ms/2)] for i in range(int(ms/2), len(xs), int(ms/2)))
#    return list(xs[i:i+ms] for i in range(0, len(xs), ms))

In [ ]:
def suma_repetidos(freq_int, sp):
    sp_int = []
    frequ_int = []
    for i in range(1,len(freq_int)):
        frequ = int(freq_int[i-1])
        frequ_1 = int(freq_int[i])
        try:
            frequ_2 = int(freq_int[i+1])
        except:
            frequ_2 = int(freq_int[-1])
        if i == 1:
            frequ_int.append(frequ)
        if frequ == frequ_1:
            try:
                sp_new = sp_int[-1] + sp[i+1]
                sp_int.pop()
                sp_int.append(sp_new)
#                print("blop")
            except:
                sp_new = sp[i] + sp[i+1]
                sp_int.append(sp_new)
#                print("blopo")
            if frequ_int[-1] != frequ_1:
                frequ_int.append(frequ_1)
        else:
            sp_new = sp[i]
            sp_int.append(sp_new)
            frequ_int.append(frequ_1)
#            print("plopo")
    return frequ_int, sp_int

In [ ]:
def exist_or_zero(freq_new,sp,freq_final):
    sp_final = []
    maxxo = max(sp)
    for freq in freq_final:
        if freq in freq_new:
            spe = sp[list(freq_new).index(freq)]
            sp_final.append(spe/maxxo)
        else:
            sp_final.append(0)
    return sp_final

In [ ]:
def extract_peaks_and_freqs(aud, Fs):
    sp = scipy.fft.fft(aud)
    freq = scipy.fft.fftfreq(len(aud),Fs)
    freq_sci= scipy.signal.find_peaks(sp)[0]
    sp_sci = scipy.signal.peak_prominences(sp, peaks=freq_sci)[0]
    freq_gaps = np.arange(0,6750,1)
    sp_final = exist_or_zero(freq_sci, sp_sci, freq_gaps)
    freq_final, _ = scipy.signal.find_peaks(sp_final)
    pikos = [sp_final[freq_final[i]] for i in range(0,len(freq_final))]
    sp_sorted = sorted(pikos, reverse= True)
    freq_sorted=[freq_final[pikos.index(pikoso)] for pikoso in sp_sorted]
    return sp_sorted, freq_sorted, sp_final, freq_final

In [ ]:
def final_data_collection(freq_sorted, pikos_sorted, n, m, note_played):
    df_final = pd.DataFrame({'peak_1': [], 'peak_2': [], 'Magnitude difference': [],'instrument': m, 'note_played': []})
    frequs_sp = []
    for i in range(0,len(freq_sorted[:n])):
        freq_1, sp_1 = freq_sorted[i], pikos_sorted[i]
        frequs_sp.append((freq_1, sp_1))
        for j in range(0,len(freq_sorted[:n])):
            freq_2, sp_2 = freq_sorted[j], pikos_sorted[j]
            if (freq_2, sp_2) in frequs_sp:
                continue
            else:
                distance = np.sqrt((freq_2 - freq_1)**2 + (sp_2 - sp_1)**2)
                df_iteration = pd.DataFrame({'peak_1':[(freq_1, sp_1)], 'peak_2':[(freq_2, sp_2)], 'Magnitude difference': [distance] ,'instrument': 1, 'note_played': [note_played]})
                df_final = df_final.append(df_iteration)
    return df_final

In [ ]:
common_path = '/home/jacs/Documents/DataScience/Personal/song_similarity_audio/TinySOL/'
df_into = pd.read_csv(common_path+'TinySOL_metadata.csv')
df_into= df_into[df_into['Needed digital retuning']]
instruments = df_into['Instrument (in full)'].unique()

In [ ]:
instruments

In [ ]:
instruments_test = ['Trombone', 'Trumpet in C']
outputs = ['database_trombone_10_peaks', 'database_trumpet_10_peaks']

In [ ]:
#audio_files = ['A-sharp-trumpet', 'B-trumpet', 'C-sharp-trumpet', 'D-sharp-trumpet', 'E-trumpet', 'F-sharp-trumpet', 'G-sharp-trumpet']

df_final = pd.DataFrame({'peak_1': [], 'peak_2': [], 'Magnitude difference': [],'instrument': [], 'note_played': []})

i=0

for woko in df_into[df_into['Instrument (in full)'] == 'Trumpet in C']['Path']:
    wavo = common_path + woko
    titulo = woko.split('/')[-1]
#    file_1 = file.format(audio_1)
#    wavo = path + file_1
    Fs, audio = wavfile.read(wavo)
    length = audio.shape[0] / Fs
    audio_chunks = chunks(audio,int(length)*2)
    print(f"length = {length}s")
    for aud in audio_chunks[2:-2]:
        length_2 = aud.shape[0] / Fs
#        print(Fs)
        # select left channel only
        try:
            aud = aud[:,0]
            print('plop')
        except:
            aud = aud[:]
            print('anti-plop')
        pikos_sorted, freq_sorted, sp_final, peaks  = extract_peaks_and_freqs(aud, Fs)
        print(i)
        i+=1
        df_final_2 = final_data_collection(freq_sorted, pikos_sorted, 10, 1, titulo).reset_index(drop=True)
        df_final = df_final.append(df_final_2).reset_index(drop=True)
df_final=df_final.reset_index(drop=True)
df_final.to_csv(output[1]+'.csv', index=False)
#################################
#        plt.specgram(aud, Fs=Fs)
#        plt.xticks(time_cnk)    
#        plt.ylim(0,5000)
#        plt.title(titulo)
#        plt.show()
#        time = np.linspace(0., length_2, aud.shape[0])
#        plt.plot(time, aud)
#        plt.title('Original signal')
#        plt.show()
#        plt.plot(peaks, [sp_final[i] for i in peaks])
#        plt.plot(freq_sorted[:10], pikos_sorted[:10],'x')
#        plt.title('Final frequencies and intensities')
#        plt.show()
#        ss = np.fft.ifft(sp_final)
#        time = np.linspace(0., length_2, len(sp_final))
#        plt.plot(time, ss)
#        plt.title('Reconstruccion with data manipulation')
#        plt.show()
#################################

In [ ]:
df_final

In [ ]:
df_final=df_final.reset_index(drop=True)

In [ ]:
df_final.to_csv('database_trumpet_10_peaks.csv', index=False)

In [ ]:
pd.read_csv('database_trumpet_freq.csv')

In [ ]:
max(freq_sci)

In [ ]:
wavfile.write("example.wav", Fs, ss.astype(np.int16))

In [ ]:
sp = scipy.fft.fft(aud)
freq = scipy.fft.fftfreq(len(aud),Fs)
freq_sci= scipy.signal.find_peaks(sp)[0]
sp_sci = scipy.signal.peak_prominences(sp, peaks=freq_sci)[0]
freq_gaps = np.arange(0,6750,1)
sp_final = exist_or_zero(freq_sci, sp_sci, freq_gaps)
freq_final, _ = scipy.signal.find_peaks(sp_final)
pikos = [sp_final[freq_final[i]] for i in range(0,len(freq_final))]
sp_sorted = sorted(pikos, reverse= True)
freq_sorted=[freq_final[pikos.index(pikoso)] for pikoso in sp_sorted]

In [ ]:
plt.plot(freq_gaps, sp_final)
plt.plot(freq_sorted[:15],sp_sorted[:15], 'x')

# Usando numpy

In [ ]:
#audio_files = ['A-sharp-trumpet', 'B-trumpet', 'C-sharp-trumpet', 'D-sharp-trumpet', 'E-trumpet', 'F-sharp-trumpet', 'G-sharp-trumpet']

df_final = pd.DataFrame({'note_played':[], 'frequencies':[], 'intensities':[], 'instrument': []})

for audio_1 in audio_files:
    file_1 = file.format(audio_1)
    wavo = path + file_1
    Fs, audio = wavfile.read(wavo + '.wav')
    length = audio.shape[0] / Fs
    audio_chunks = chunks(audio,int(length)*2)
    print(f"length = {length}s")
    for aud in audio_chunks[2:-2]:
#        print(Fs)
        # select left channel only
        try:
            aud = aud[:,0]
            print('plop')
        except:
            aud = aud[:]
            print('anti-plop')
        # trim the first 125 seconds
        sp_final = np.fft.fft(aud)
        freq_final = np.fft.fftfreq(aud.shape[-1], d=(1/Fs))
        plt.specgram(aud, Fs=Fs)
        plt.ylim(0,5000)
        time = np.linspace(0., length, aud.shape[0])
        freq_positive = [(freq_final[i], sp_final[i]) for i in range(0,len(freq_final)) if freq_final[i]>=0]
        freq_final = [freq_positive[i][0] for i in range(0,len(freq_positive))]
        sp_final = [freq_positive[i][1] for i in range(0,len(freq_positive))]
        plt.title(file_1)
        plt.show()
        plt.plot(time, aud)
        plt.title('Original signal')
        plt.show()
        plt.plot(freq_final,sp_final)
        plt.title('Final frequencies and intensities')
        plt.show()
        ss = np.fft.ifft(sp_final)
        time = np.linspace(0., length, len(sp_final))
        plt.plot(time, ss)
        plt.title('Reconstruccion with data manipulation')
        plt.show()
        df_iteration = pd.DataFrame({'note_played': file_1, 'frequencies':[freq_final], 'intensities':[sp_final], 'instrument': 1})
        df_final = df_final.append(df_iteration)

In [ ]:
df_final

# Usando spectrogram de matplotlib

In [ ]:
for audio in audio_files:
    file_1 = file.format(audio_1)
    wavo = path + file_1
    Fs, audio = wavfile.read(wavo + '.wav')
    length = audio.shape[0] / Fs
    audio_chunks = chunks(audio,int(length*8))
    print(f"length = {length}s")
    for aud in audio_chunks[1:-1]:
        # select left channel only
        try:
            aud = aud[:,0]
            print('plop')
        except:
            aud = aud[:]
            print('anti-plop')
        # trim the first 125 seconds
        length = aud.shape[0] / Fs
        first = aud[:int(Fs*length)]
        powerSpectrum, frequenciesFound, time2, imageAxis = plt.specgram(first, Fs=Fs)
        plt.ylim(0,5000)
        time = np.linspace(0., length, aud.shape[0])
        plt.title(file_1)
        plt.show()
        plt.plot(time, aud)
        plt.show()
        print(frequenciesFound)
    #    poweroso = [np.average(powerSpectrum[i,:]) for i in range(0,len(powerSpectrum))]
    #    print(len(poweroso))
    #    ss = np.fft.ifft(poweroso)
        plt.plot(frequenciesFound,powerSpectrum)
        plt.xlim(0,2500)
        plt.title('frequencies')
        plt.show()
    #    plt.plot(ss.real)
    #    plt.title('Reconstruccion')
    #    plt.show()
    #    plt.savefig('{}.png'.format(audio))

In [ ]:
len(frequenciesFound)

In [ ]:
powerSpectrum